In [ ]:
import os
import pandas as pd
import joblib
import requests

In [ ]:
DATA_OUTPUT_DIR = os.path.join('..','00.data','output')

In [ ]:
if "DATA_OUTPUT_DIR" in os.environ:
    DATA_OUTPUT_DIR = os.environ["DATA_OUTPUT_DIR"]
if "PYAUTOTRADER_URL" not in os.environ:
    raise EnvironmentError("PYAUTOTRADER_URL not defined")
else:
    PYAUTOTRADER_URL = os.environ["PYAUTOTRADER_URL"]

In [ ]:
files_found = [x for x in os.listdir(DATA_OUTPUT_DIR) if x.endswith('.pickle')]
files_found_tokens=[x.split('.') for x in files_found]

models_found = {}

for current_model in files_found_tokens:
    model_name = '.'.join(current_model[:-2])
    filename = os.path.join(DATA_OUTPUT_DIR,'.'.join(current_model))
    if not model_name in models_found:
        models_found[model_name]= {}          
    models_found[model_name][current_model[-2]]=filename

In [ ]:
first_model = list(models_found.keys())[0]
current_parameters = joblib.load(models_found[first_model]['parameters'])

In [ ]:
CURRENT_EXCHANGE = current_parameters['CURRENT_EXCHANGE']
CURRENT_ASSET = current_parameters['CURRENT_ASSET']
CURRENT_TIMEFRAME= current_parameters['CURRENT_TIMEFRAME']
CURRENT_TARGET= current_parameters['CURRENT_TARGET']
CURRENT_STOP= current_parameters['CURRENT_STOP']

In [ ]:
predict_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.predicts.xlsx")
predict_full_file_name = os.path.join(DATA_OUTPUT_DIR,predict_file_name)
dfPredict = pd.read_excel(predict_full_file_name)

In [ ]:
def predict_short(x):
    url_final = f'{PYAUTOTRADER_URL}predict/{CURRENT_EXCHANGE}/{CURRENT_ASSET}/{CURRENT_TIMEFRAME}/{int(x["current_date"])}/{int(x["current_time"])}/'
    return requests.get(url_final).json()['short_predict']

def predict_long(x):
    url_final = f'{PYAUTOTRADER_URL}predict/{CURRENT_EXCHANGE}/{CURRENT_ASSET}/{CURRENT_TIMEFRAME}/{int(x["current_date"])}/{int(x["current_time"])}/'
    return requests.get(url_final).json()['long_predict']


In [ ]:
dfPredict['short_predict_server'] = dfPredict.apply( lambda x: predict_short(x), axis=1)
dfPredict['long_predict_server'] = dfPredict.apply( lambda x: predict_long(x), axis=1)

In [ ]:
dfPredict['diff_predict_short'] = dfPredict.apply( lambda x: x['short_predict_server'] - x['short_predict'] , axis=1)
dfPredict['diff_predict_long'] = dfPredict.apply( lambda x: x['long_predict_server'] - x['long_predict'], axis=1)

In [ ]:
predict_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.predicts_server.xlsx")
predict_full_file_name = os.path.join(DATA_OUTPUT_DIR,predict_file_name)

dfPredict.to_excel(predict_full_file_name)